(from last class) Real world example

https://github.com/pytorch/examples/blob/main/imagenet/main.py


In [ ]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

In [ ]:
%pwd

In [ ]:
%cd /content/drive/MyDrive/dlss24

In [ ]:
%pwd

# Data

Download the corpora data from: https://codeocean.com/capsule/0078777/tree/v1

Uploaded to a location outside your gitHub (to do not be tracked)

You pwd should have the files:


1.   source_corpus.csv
2.   target_corpus.csv




# Module IV:  - Class 6 - Text Classifiers

This notebook have the network framework pytorch and how CNNs work

# Basic Pre processing and tokenization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
#load the pickle file in OLAT
#df_dummies_model.to_pickle('df_dummies_model.pkl')

df_source_corpus=pd.read_csv('/content/drive/MyDrive/dlss24/source_corpus.csv')

In [ ]:
df_source_corpus.head()



> Create a plot with the number of sentences (or rows) per topic



In [ ]:
#### YOUR CODE STARTS HERE ####
from matplotlib import pyplot as plt
import seaborn as sns

...

# Add labels and title if needed
plt.xlabel('Count')
plt.ylabel('Topic')
plt.title('Number of Sentences per Topic')

# Show the plot
plt.show()


##### YOUR CODE ENDS HERE ####

## Sentence Tokenization

In [ ]:
text = "K. Alex Müller on the right track: Soon afterwards, he and M.A's J. Georg Bednorz discovered the first high-temperature superconductor. The 2 are nobel prize winners."

**NLTK**: fast with some errors

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

In [ ]:
# by puntuation
sentences = sent_tokenize(text) # split document into sentences
print(sentences)

**Spacy**: better than NLTK

In [ ]:
import spacy
# by language
nlp = spacy.load('en_core_web_sm')


In [ ]:
doc = nlp(text)
sentences = list(doc.sents)
print(sentences)

In [ ]:
doc.is_sentenced

# Pre Processing



0.   Choose the unit (sentence, document, paragraph, speech)

1.   Capitalization
2.   Puntuation
3.   Tokens
4.   Numbers
5.   Stopwords
6.   Stemming
7.   Lemmatizing



### 1. Capitalization

In [ ]:
# Capitalization
text_lower = text.lower() # go to lower-case

In [ ]:
text_lower

### 2. Punctuation

In [ ]:
# recipe for fast punctuation removal
from string import punctuation
print ("punctuation:", punctuation)

In [ ]:
#maktrans function from str, args are: find-character, replace-character, remove-character
punc_remover = str.maketrans('','',punctuation)
text_nopunc = text_lower.translate(punc_remover)
print(text_nopunc)

### 3. Tokenization of words

In [ ]:
# Tokens
tokens = text_nopunc.split() # splits a string on white space
print(tokens)

### 4. Numbers



> Replace the number found by the # character



In [ ]:
#### YOUR CODE STARTS HERE ####



# keep if not a digit, else replace with "#"
norm_numbers = ...
print(no_numbers )
print(norm_numbers)



##### YOUR CODE ENDS HERE ####

### 5. Stopwords

From NLTK

In [ ]:
# Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
stoplist = stopwords.words('english')
print ("stop words:", stoplist)

In [ ]:
# keep if not a stopword
nostop = [t for t in norm_numbers if t not in stoplist]
print(nostop)

From scikit-learn

In [ ]:
# scikit-learn stopwords
# depending on sklearn version, for sklearn==0.24.1, stop_words are here
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS as stop_words
sorted(list(stop_words))[:20]

In [ ]:
# spacy stopwords
sorted(list(nlp.Defaults.stop_words))[:20]

### 6. Stemming

Heuristic based, reduce variations of words like -ed, -s

In [ ]:
# Stemming
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english') # snowball stemmer, english

In [ ]:
# remake list of tokens, replace with stemmed versions
tokens_stemmed = [stemmer.stem(t) for t in tokens]
print(tokens_stemmed)

### 7. Lemmatizing

Linguistic-based, reduces words with linguistic meaning. Superlatives and Verbs infinitive/base form.

In [ ]:
# Lemmatizing
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

In [ ]:
wnl = WordNetLemmatizer()
wnl.lemmatize('corporation'), wnl.lemmatize('corporations')

In [ ]:
stemmer.stem('corporations')

In [ ]:
wnl.lemmatize('was')

In [ ]:
wnl.lemmatize('went') #go

In [ ]:
wnl.lemmatize('better') #good

All in one function:

In [ ]:
from string import punctuation
translator = str.maketrans('','',punctuation)
from nltk.corpus import stopwords
stoplist = set(stopwords.words('english'))
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

def normalize_text(doc):
    "Input doc and return clean list of tokens"
    doc = doc.replace('\r', ' ').replace('\n', ' ')
    lower = doc.lower() # all lower case
    nopunc = lower.translate(translator) # remove punctuation
    words = nopunc.split() # split into tokens
    nostop = [w for w in words if w not in stoplist] # remove stopwords
    no_numbers = [w if not w.isdigit() else '#' for w in nostop] # normalize numbers
    stemmed = [stemmer.stem(w) for w in no_numbers] # stem each word
    return stemmed



> Apply the normalized_text function to the text column of df_source_corpus




In [ ]:
#### YOUR CODE STARTS HERE ####


df_source_corpus['tokens_cleaned'] = ...




##### YOUR CODE ENDS HERE ####

**Gensim**: All in one

In [ ]:
from gensim.utils import simple_preprocess # lowercase, tokenized, punctuations/numbers removed
print(simple_preprocess(text))

In [ ]:
# I have cleaned 'text' colun before
df_source_corpus['tokens_cleaned'] = df_source_corpus['text'].apply(simple_preprocess)
df_source_corpus['tokens_cleaned']

(OPTIONAL) If you want to customize the gensim:

In [ ]:
complicated_text = "<div>Prof. Zurich <i>hailed</i> from Zurich., She got 3 M.A.'s from ETH.</div>" # added html tags

from gensim.parsing.preprocessing import preprocess_string, \
strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short, stem_text

preprocess_string(complicated_text) # with all default filters: lowercase, tags, puncations, whitespaces, numerics, short words, stop words.

# remove some filters to keep more tokens.
# e.g. If I want to keep stopwords, short words, and numbers and keep the words not stemmed
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces]
preprocess_string(complicated_text, CUSTOM_FILTERS) # only use the customized filters

# Full analysis with Spacy

In [ ]:
dfs = df_source_corpus.sample(1000)
dfs['doc'] = dfs['text'].apply(nlp)

In [ ]:
dfs=dfs.reset_index()

In [ ]:
# The spacy model already gives you sentences and tokens.
# For example:
sent1 = list(dfs['doc'].iloc[0].sents)[0]
sent1 # sentence

In [ ]:
# tokens
list(sent1)

In [ ]:
# lemmas
[x.lemma_ for x in sent1]

In [ ]:
# POS tags
[x.tag_ for x in sent1]

In [ ]:

from collections import Counter
# Initialize a Counter to count tokens
token_counter = Counter()

# Iterate over each Doc object in the 'doc' column and count tokens
for doc in dfs['doc']:
    token_counter.update([token.text for token in doc])

# Print the token counts
print(token_counter)

In [ ]:
token_counter.most_common()[:20]



> Print the 20 least common tokens




In [ ]:
#### YOUR CODE STARTS HERE ####


 ...




##### YOUR CODE ENDS HERE ####

In [ ]:
dfs['num_words'] = dfs['doc'].apply(lambda x: len(list(x)))
dfs['num_sents'] = dfs['doc'].apply(lambda x: len(list(x.sents)))


print(len(token_counter),'words in corpus.')
words_per_sent = len(token_counter) / len(dfs['doc'])
print(words_per_sent,'words per sentence.')

In [ ]:
dfs

[CountVectorizer Doc](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=0.01, # at min 1% of docs
                        max_df=.9,
                        max_features=1000,
                        stop_words='english',
                        ngram_range=(1,3))
X = vec.fit_transform(dfs['text'])

# save the vectors
#pd.to_pickle(X,'X.pkl')





> Print the shape of the Count Vectorizer and one of its samples



In [ ]:
#### YOUR CODE STARTS HERE ####


 ...




##### YOUR CODE ENDS HERE ####

In [ ]:
# Select tokens with counts greater than 300
selected_tokens = {token: count for token, count in token_counter.items() if 12< count < 90 }


In [ ]:
len(selected_tokens)

Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency.


In [ ]:
# tf-idf vectorizer up-weights rare/distinctive words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=0.01,
                        max_df=0.9,
                        max_features=1000,
                        stop_words='english',
                        use_idf=True, # the new piece
                        ngram_range=(1,2),
                        smooth_idf=True)

X_tfidf = tfidf.fit_transform(dfs['text'])
#pd.to_pickle(X_tfidf,'X_tfidf.pkl')

In [ ]:
X_tfidf[0].toarray()

In [ ]:
dfs['topic_8'].value_counts()

In [ ]:
vocab = tfidf.get_feature_names_out()
vocab[:10], vocab[-10:]

### Word Cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

for topic_id in ['welfare and quality of life','economy','social groups','political system']:
    slicer = dfs['topic_8'] == topic_id

    #selecting rows
    f = X_tfidf[slicer.values]

    #sum of freq per row
    total_freqs = list(np.array(f.sum(axis=0))[0])

    #put vocab and frew to feed Wordcloud
    fdict = dict(zip(vocab,total_freqs))
    # generate word cloud of words with highest counts
    wordcloud = WordCloud().generate_from_frequencies(fdict)
    print(topic_id)
    plt.clf()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()



> Use the replication material to build a Political Science Topic Classifier

[Topic Classification for Political Texts](https://www.cambridge.org/core/journals/political-analysis/article/crossdomain-topic-classification-for-political-texts/F074564984969CE168BCBCF5E7D931C8#article)


*What could they have done different in the pre process step?*

In [ ]:
#### YOUR CODE STARTS HERE ####


 ...




##### YOUR CODE ENDS HERE ####

In [ ]:
#This was in 2019 (print the metrics)
metrics

In [ ]:
#This was in 2023
# https://manifesto-project.wzb.eu/information/documents/manifestoberta
